Staffing levels of nursing homes

In [1]:
import pandas as pd

Provider info dataset: https://data.medicare.gov/Nursing-Home-Compare/Provider-Info/4pq5-n9py

CMS minimum standards: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4833431/

In [2]:
df = pd.read_csv(
    "../data/staffing.csv",
    parse_dates = ["processing_date"]
)

What percent of all nursing homes are below the CMS guidelines for staffing?

In [3]:
# Remove facilities with footnotes that indicate unreliable staffing data
verified = (
    df
    .loc[
        lambda x: ~x["staffing_rating_footnote"].isin([1, 2, 12, 18])
    ]
)

In [4]:
# Pre calculate
BELOW_CALCS = dict(
    below_total = lambda x: x["reported_total_nurse_staffing_hours_per_resident_per_day"] < 4.1,
    below_rn = lambda x: x["reported_rn_staffing_hours_per_resident_per_day"] < 0.75,
    below_lpn = lambda x: x["reported_lpn_staffing_hours_per_resident_per_day"] < 0.55,
    below_nurse_aid = lambda x: x["reported_cna_staffing_hours_per_resident_per_day"] < 2.8,
)

provider_tests = (
    verified
    .assign(**BELOW_CALCS)
    [[
        "provider_state",
        "federal_provider_number",
    ] + list(BELOW_CALCS.keys())]
)

provider_tests.head()

,provider_state,federal_provider_number,below_total,below_rn,below_lpn,below_nurse_aid
0,AL,015407,True,True,True,True
3,CT,075322,False,False,True,False
5,FL,105393,False,True,False,True
7,GA,115404,True,True,False,True
8,GA,115533,True,True,False,True


In [5]:
(
    provider_tests
    [list(BELOW_CALCS.keys())]
    .mean()
)

below_total        0.701722
below_rn           0.684927
below_lpn          0.163773
below_nurse_aid    0.848204
dtype: float64

By state

In [6]:
(
    provider_tests
    .groupby([ "provider_state" ])
    [list(BELOW_CALCS.keys())]
    .mean()
    .sort_values("below_total", ascending = False)
    .round(2)
    .head(10)
)

,below_total,below_rn,below_lpn,below_nurse_aid
provider_state,,,,
TX,0.88,0.93,0.02,0.94
GA,0.86,0.96,0.00,0.97
MO,0.85,0.91,0.26,0.89
OK,0.84,0.99,0.05,0.85
LA,0.84,0.97,0.01,0.92
NM,0.83,0.68,0.54,0.86
IN,0.83,0.70,0.09,0.93
RI,0.82,0.33,0.82,0.78
IL,0.81,0.62,0.38,0.90


---

---

---